# Notebook Imports

In [2]:
import pandas as pd
import numpy as np

# Constants

In [81]:
# Path for training & test data file in .txt format
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

# Path for testing data files in .txt format
TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

# Path for token probabilities in .txt format
TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

# Used Vocabulary Size for classificator
VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [4]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [6]:
sparse_train_data[:5]

array([[ 0,  0,  1,  2],
       [ 0,  7,  1,  1],
       [ 0, 16,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 24,  1,  1]])

In [7]:
sparse_train_data[-5:]

array([[5795, 1825,    0,    1],
       [5795, 1892,    0,    2],
       [5795, 2116,    0,    1],
       [5795, 2273,    0,    1],
       [5795, 2454,    0,    1]])

In [8]:
# Checking number of rows for both files
print('Number of rows in training file', sparse_train_data.shape[0])
print('Number of rows in test file', sparse_test_data.shape[0])

Number of rows in training file 265577
Number of rows in test file 110579


In [9]:
# Checking number of emails in files
# The comma separets the rows from the columns inside the square brackets
print('Number of emails in training file', np.unique(sparse_train_data[:, 0]).size)
print('Number of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Number of emails in training file 4014
Number of emails in test file 1723


## Creating an Empty DataFrame

In [10]:
# Defining columns for empty dataframe
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
# Defining indices 
index_names = np.unique(sparse_train_data[:, 0])
index_names[:10]

array([ 0,  1,  2,  3,  4,  6,  7,  9, 10, 11])

In [13]:
# Creating an empty dataframe
full_train_data = pd.DataFrame(index=index_names, columns=column_names)

# Using fillna function to replace NaN with zeroes
full_train_data.fillna(value=0, inplace=True)

In [14]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Creating a Full Matrix from a Sparse Matrix

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix --- Numpy array
    nr_words --- Size of the vocabulary. Total number of tokens
    doc_idx --- Position of the document id in the sparse matrix. Default: 1st column
    word_idx --- Position of the word id in the sparse matrix. Default: 2nd column
    cat_idx --- Position of the label (spam is 1, nonspam is 0). Default 3rd column
    freq_idx --- Position of occurrence of word in sparse matrix. Default 4th column
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    # Filling the full sparse matrix with a for loop
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence  
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [51]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 32.3 s, sys: 1.61 s, total: 33.9 s
Wall time: 36.2 s


In [17]:
full_train_data.head(10)

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,1,0,1,0,4,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,1,0,0,2,1,1,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,4,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,1,0,2,2,5,3,5,1,4,3,...,0,0,0,0,0,0,0,0,0,0


# Training Naive Bayes Model

## Calculating the probability of Spam

In [18]:
# Calculating the probability of spam - The percent of spam messages in the training dataset, stored in
# a variable called prob_spam

spam = full_train_data[full_train_data['CATEGORY'] == 1]

prob_spam = round((spam.shape[0] / full_train_data['CATEGORY'].size * 100), 4 ) 
print(f'The probability of encounter an spam email among the training test is: {prob_spam}%')

# Alternative way as presented on the course:
# full_train_data.CATEGORY.size
# full_train_data.sum()
# prob_spam = full_train_data.sum() / full_train_data.CATEGORY.size

The probability of encounter an spam email among the training test is: 31.1161%


## Total Number of Words / Tokens

In [19]:
# Creating a subset that will hold all columns except 'CATEGORY' column
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Summing the number of words/tokens for each email
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4014,)

In [21]:
# Number of words/tokens for each email
email_lengths[:5]

DOC_ID
0     50
1     76
2     87
3     76
4    137
dtype: int64

In [22]:
# Calculating the total number of words/tokens in subset. 'wc' stands for 'word count'.
total_wc = email_lengths.sum()
total_wc

444948

## Number of Tokens in Spam & Ham Emails

In [23]:
# Creating a subset of the email series that only contains spam messages, spam_lengths. 
#   Counting total number of words in spam emails
# Creating another subset called ham_lengths and counting words

spam_lengths = full_train_data[full_train_data['CATEGORY'] == 1].sum(axis=1)
spam_wc = spam_lengths.sum()
spam_wc

195217

In [24]:
ham_lengths = full_train_data[full_train_data['CATEGORY'] == 0].sum(axis=1)
ham_wc = ham_lengths.sum()
ham_wc

250980

In [25]:
# Checking if proportion of ham and spam subsets corresponds to dataset length 
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [26]:
# Checking if emails from spam tend to be longer than ham emails
# Checking which category is longer
spam_lengths.shape[0] < ham_lengths.shape[0]

True

In [27]:
# On average spam emails have around 156 words per each email
spam_lengths.mean()

156.2986389111289

In [28]:
# On average ham emails have around 90 words per each email, they are shorter than spam emails
ham_lengths.mean()

90.77034358047017

In [29]:
# As proposed in the course
print('Average number of words in spam emails {:.3f}', format(spam_wc / spam_lengths.shape[0]))
print('Average number of words in ham emails {:.3f}', format(ham_wc / ham_lengths.shape[0]))

Average number of words in spam emails {:.3f} 156.2986389111289
Average number of words in ham emails {:.3f} 90.77034358047017


## Summing the Tokens Ocurring in Spam

In [30]:
full_train_features.shape

(4014, 2500)

In [31]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY ==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [32]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,2,0,0,3,2,2,2,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1887,0,0,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,2,0,1,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,3,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1895,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
train_spam_tokens.shape

(1249, 2500)

In [36]:
# By default sum() method axis is 0
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [37]:
summed_spam_tokens.tail()

2495    28
2496     7
2497     3
2498    10
2499     7
dtype: int64

In [39]:
# Summing tokens for ham emails
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY ==0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens.shape

(2500,)

In [40]:
summed_ham_tokens.tail()

2495     3
2496    22
2497    29
2498    17
2499    32
dtype: int64

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [41]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.009008
1    0.004896
2    0.006838
3    0.010596
4    0.006873
dtype: float64

In [42]:
prob_tokens_spam.sum()

0.9936828901915364

## P(Token | Ham) - Probability that a Token Occurs given the Email is Ham

In [43]:
prob_tokens_ham = summed_ham_tokens / (ham_wc + VOCAB_SIZE)
prob_tokens_ham.sum()

1.0

## P(Token) - Probability that Token Occurs

In [44]:
prob_tokens_all = full_train_features.sum() / total_wc
prob_tokens_all.sum()

1.0

## Save the Trained Model

In [49]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

## Prepare Test Data

In [56]:
%%time
# Creating a full matrix for test data%%time
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

CPU times: user 10.9 s, sys: 176 ms, total: 11.1 s
Wall time: 11.3 s


In [82]:
# Creating a subset that will hold all columns except 'CATEGORY' column
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [83]:
# Saving 
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)